In [265]:
# This tells matplotlib not to try opening a new window for each plot.
%matplotlib inline

# General libraries
import time
import pandas as pd
import re
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import MultipleLocator

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn library for importing the newsgroup data.
from sklearn.datasets import fetch_20newsgroups

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

# Set the randomizer seed so results are the same each time.
np.random.seed(0)

data = pd.read_csv(r"data\covtype.csv")
train_df = pd.read_csv(r"data\train.csv")
test_df = pd.read_csv(r"data\test.csv")

# Build the np arrays
train_data = train_df.to_numpy()
test_data = test_df.to_numpy()

# Get last column for train labels
train_labels = train_data[ :,55]

#Remove last column from train_data because that is the labels
train_data = np.delete(train_data, 55, axis=1)

# Shuffle the input
shuffle = np.random.permutation(np.arange(train_data.shape[0]))
train_data, train_labels = train_data[shuffle], train_labels[shuffle]

# Set some variables to hold test, dev, and training data.
#test_data, test_labels = train_data[0:2000,:], train_labels[0:2000]
dev_data, dev_labels = train_data[2000:14000,:], train_labels[2000:14000]
mini_train_data, mini_train_labels = train_data[14000:15000,:], train_labels[14000:15000]

In [276]:
#Build new datasets for converting Soil_Types and Wilderness_Areas

def binary_to_nominal(old_array):

    column_to_be_added = np.ones(old_array.shape[0],int)
    new_data = np.column_stack((old_array, column_to_be_added))

    for i,row in enumerate(old_array):

        new_data[i][new_data.shape[1]-1] = -1
        soil_type = 0

        for j, col in enumerate(data_columns):
            if col[0:9] == "Soil_Type":
                if new_data[i][j] == 1:
                    soil_type = int(col[9:11])
                    break

        if soil_type > 0:
            new_data[i][new_data.shape[1]-1] = soil_type
        else:
            print("error")

    cols_to_delete = []
    for j, col in enumerate(data_columns):
        if col[0:9] == "Soil_Type":
            cols_to_delete.append(j)
    new_data = np.delete(new_data, cols_to_delete, axis=1)



    column_to_be_added = np.ones(new_data.shape[0],int)
    new_data = np.column_stack((new_data, column_to_be_added))

    for i,row in enumerate(old_array):

        new_data[i][new_data.shape[1] - 1] = -1
        wilderness_area = 0

        for j, col in enumerate(data_columns):     
            if col[0:15] == "Wilderness_Area":
                if new_data[i][j] == 1:
                    wilderness_area = int(col[15])
                    break

        if wilderness_area > 0:
            new_data[i][new_data.shape[1]-1] = wilderness_area
        else:
            print("error")

    cols_to_delete = []
    for j, col in enumerate(data_columns):
        if col[0:15] == "Wilderness_Area":
            cols_to_delete.append(j)
    new_data = np.delete(new_data, cols_to_delete, axis=1)

    return new_data


# Build the np arrays
train_data2 = train_df.to_numpy()
test_data2 = test_df.to_numpy()

# Get last column for train labels
train_labels2 = train_data2[ :,55]

#Remove last column from train_data because that is the labels
train_data2 = np.delete(train_data2, train_data2.shape[1]-1, axis=1)

train_data2 = binary_to_nominal(train_data2)
test_data2 = binary_to_nominal(test_data2)

# Shuffle the input
shuffle = np.random.permutation(np.arange(train_data2.shape[0]))
train_data2, train_labels2 = train_data2[shuffle], train_labels2[shuffle]

# Set some variables to hold test, dev, and training data.
#test_data2, test_labels2 = train_data2[0:2000,:], train_labels2[0:2000]
dev_data2, dev_labels2 = train_data2[2000:14000,:], train_labels2[2000:14000]
mini_train_data2, mini_train_labels2 = train_data2[14000:15000,:], train_labels2[14000:15000]

## Decision Tree

In [266]:
from sklearn.model_selection import cross_val_score
from sklearn import tree

def print_importances(clf):
    importances = np.round(clf.feature_importances_,4)
    features = train_df.columns[0:55].to_numpy()
    importances_df = pd.DataFrame({'feature':features,'importance':importances})
    importances_df = importances_df.sort_values('importance',ascending=False)
    importances_df = importances_df[(importances_df.sum(axis=1) != 0)]  
    print(importances_df)
    
def print_structure(clf):
    n_nodes = clf.tree_.node_count
    children_left = clf.tree_.children_left
    children_right = clf.tree_.children_right
    feature = clf.tree_.feature
    threshold = clf.tree_.threshold

    node_depth = np.zeros(shape=n_nodes, dtype=np.int64)
    is_leaves = np.zeros(shape=n_nodes, dtype=bool)
    stack = [(0, 0)]  # start with the root node id (0) and its depth (0)
    while len(stack) > 0:
        # `pop` ensures each node is only visited once
        node_id, depth = stack.pop()
        node_depth[node_id] = depth

        # If the left and right child of a node is not the same we have a split
        # node
        is_split_node = children_left[node_id] != children_right[node_id]
        # If a split node, append left and right children and depth to `stack`
        # so we can loop through them
        if is_split_node:
            stack.append((children_left[node_id], depth + 1))
            stack.append((children_right[node_id], depth + 1))
        else:
            is_leaves[node_id] = True

    print("The binary tree structure has {n} nodes and has the following tree structure:\n".format(n=n_nodes))
    for i in range(n_nodes):
        if is_leaves[i]:
            values = clf.tree_.value[i]
            max_value = values.max()
            cover_type = np.argmax(values)
            print("{space}node={node} is a leaf node with max value of {value} and cover type of {ctype}"
                  .format(space=node_depth[i] * "\t", node=i, value=max_value, ctype=cover_type))
        else:
            print("{space}node={node} is a split node: go to node {left} if {feature} <= {threshold} else to node {right}.".format(
                      space=node_depth[i] * "\t",
                      node=i,
                      left=children_left[i],
                      feature=train_df.columns[feature[i]],
                      threshold=threshold[i],
                      right=children_right[i]))



In [272]:
clf = tree.DecisionTreeClassifier(max_depth=4, random_state=0)
clf = clf.fit(train_data, train_labels)
predicted_labels = clf.predict(dev_data)
tree_model_score = clf.score(dev_data, dev_labels)
depth = clf.get_depth()
num_leaves = clf.get_n_leaves()

print('Desicion Tree Score: %.2f%%' % (tree_model_score*100))   
print('Decision Tree Depth is %d and has %d leaves' % (depth, num_leaves))

#plt.figure(figsize=(75,10))
#tree.plot_tree(clf, feature_names=train_df.columns, proportion=True)
#plt.show()
#print_importances(clf)
#print_structure(clf)
print("Confusion Matrix:\n")
print(confusion_matrix(dev_labels, predicted_labels))



### With new data
clf2 = tree.DecisionTreeClassifier(max_depth=4, random_state=0)
clf2 = clf2.fit(train_data2, train_labels2)
predicted_labels = clf2.predict(dev_data2)
tree_model_score = clf2.score(dev_data2, dev_labels2)
depth = clf2.get_depth()
num_leaves = clf2.get_n_leaves()

print('Desicion Tree Score: %.2f%%' % (tree_model_score*100))   
print('Decision Tree Depth is %d and has %d leaves' % (depth, num_leaves))
print("Confusion Matrix:\n")
print(confusion_matrix(dev_labels2, predicted_labels))

Desicion Tree Score: 64.74%
Decision Tree Depth is 4 and has 16 leaves
Confusion Matrix:

[[1068  286    0    0  148    6  187]
 [ 420  641    4    1  569   73   13]
 [   0    1  717  326  112  565    0]
 [   0    0   92 1529    0  103    0]
 [   0   81    7    0 1530   94    0]
 [   0    8  407  206  144  933    0]
 [ 371    1    0    0    6    0 1351]]
Desicion Tree Score: 66.03%
Decision Tree Depth is 4 and has 16 leaves
Confusion Matrix:

[[1233   78    1    0  143    5  244]
 [ 740  339   17    1  567   60   28]
 [   0    1 1069  307  104  208    0]
 [   0    0  198 1516    0    9    0]
 [  21  101   48    0 1503   58    0]
 [   0    8  623  213  142  716    0]
 [ 146    0    0    0    6    0 1547]]


In [273]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import plot_confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

depths = {"max_depth": [3, 4, 5, 10, 25, 30]}

#Grid search is not working as expected.  Keeps claming the second to last is better even though it is not...
grid_search_decision_tree = GridSearchCV(tree.DecisionTreeClassifier(), depths, scoring='accuracy')
grid_search_decision_tree.fit(train_data, train_labels)
predicted = grid_search_decision_tree.predict(dev_data)
optimal_depth = grid_search_decision_tree.best_params_['max_depth']
print("The optimal value for depth using GridSearchCV method is {depth} with accuracy of {accuracy}"
      .format(depth=optimal_depth, accuracy=metrics.accuracy_score(dev_labels, predicted)))

clf_final = tree.DecisionTreeClassifier(max_depth=25, random_state=0)
clf_final = clf_final.fit(train_data, train_labels)
test_predictions = clf_final.predict(test_df)

predictions_dt = pd.DataFrame(data = test_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_dt.to_csv("dt_predictions.csv")  #Kaggle Score of 0.59266
predictions_dt 

The optimal value for depth using GridSearchCV method is 25 with accuracy of 0.9978333333333333


,Cover_Type
Id,
15121,5
15122,5
15123,5
15124,5
15125,5
...,...
581008,3
581009,3
581010,3


In [278]:
# Repeat on new data set
depths = {"max_depth": [3, 4, 5, 10, 25, 30]}

# #Grid search is not working as expected.  Keeps claming the second to last is better even though it is not...
# grid_search_decision_tree = GridSearchCV(tree.DecisionTreeClassifier(), depths, scoring='accuracy')
# grid_search_decision_tree.fit(train_data2, train_labels2)
# predicted = grid_search_decision_tree.predict(dev_data2)
# optimal_depth = grid_search_decision_tree.best_params_['max_depth']
# print("The optimal value for depth using GridSearchCV method is {depth} with accuracy of {accuracy}"
#       .format(depth=optimal_depth, accuracy=metrics.accuracy_score(dev_labels2, predicted)))

clf_final = tree.DecisionTreeClassifier(max_depth=30, random_state=0)
clf_final = clf_final.fit(train_data2, train_labels2)
test_predictions = clf_final.predict(test_data2)

predictions_dt = pd.DataFrame(data = test_predictions, index = test_df.loc[:, "Id"], columns = ["Cover_Type"])
predictions_dt.to_csv("dt_predictions.csv")  #Kaggle Score of 0.59169
predictions_dt 

,Cover_Type
Id,
15121,2
15122,1
15123,2
15124,2
15125,2
...,...
581008,3
581009,3
581010,3


In [8]:
from sklearn.ensemble import RandomForestClassifier
rnd = RandomForestClassifier()
rnd.fit(train_data,train_labels)
print(cross_val_score(rnd,dev_data,dev_labels,cv = 3).mean())


from sklearn.ensemble import ExtraTreesClassifier
from  sklearn.ensemble import AdaBoostClassifier
ex = ExtraTreesClassifier(random_state  =123,n_estimators = 1000)
ex1 = AdaBoostClassifier(base_estimator = ex,n_estimators = 5)

0.8567499999999999
